##### Installing libraries

##### Calling libraries

##### Code

In [ ]:
pathToChromedriver = '/home/iazzari/repositorios/scripts/scripts_investimento/chromedriver'
DRIVER = webdriver.Chrome(pathToChromedriver)

def get_historic_values(ticker, df_historical_values):
    
    #entrando no site com o ticker q eu vou pegar os dados
    DRIVER.get('https://br.investing.com/search/?q={}&tab=quotes'.format(ticker))
    sleep(5)


    try:
        closePopup = DRIVER.find_element_by_class_name('popupCloseIcon largeBannerCloser')
        closePopup.click()
    except:
        pass

    #pegando novo link que chega ate os dados historicos
    continue_link = DRIVER.find_element_by_xpath('//*[@id="fullColumn"]/div/div[3]/div[3]/div/a')
    next_link = continue_link.get_attribute('href')

    DRIVER.get(next_link + '-historical-data')

    sleep(5)


    try:
        close_popup = DRIVER.find_element_by_class_name("popupCloseIcon largeBannerCloser")
        close_popup.click()

    except:
        pass


    try:
        date_input = DRIVER.find_element_by_id("widgetFieldDateRange")
        DRIVER.execute_script("arguments[0].click();", date_input)


    except:
        close_popup = DRIVER.find_element_by_class_name("popupCloseIcon largeBannerCloser")
        close_popup.click()
        date_input = DRIVER.find_element_by_id("widgetFieldDateRange")
        date_input.click()


    start_date_input = DRIVER.find_element_by_id("startDate")

    start_date_input.clear()
    start_date_input.send_keys(start_date)

    end_date_input = DRIVER.find_element_by_id("endDate")

    end_date_input.clear()
    end_date_input.send_keys(end_date)

    aplicar_button = DRIVER.find_element_by_id("applyBtn")
    aplicar_button.send_keys(Keys.ENTER)


    #nome das colunas do dataframe
    rows_headers = ['Data', 'Ultimo']
    
    sleep(2)
    
    table_id = DRIVER.find_element_by_id("curr_table")


    rows_table = table_id.find_element_by_tag_name("tbody")
    rows_table = rows_table.text.replace('R$','').replace('%','').replace('%','').replace('.','').replace(',','.').split('\n')

    row_content = []

    #pegando por dentro da linha e iterando nas futuras colunas
    for row in rows_table:
        values = row.split(' ')
        values[0] = datetime.datetime.strptime(values[0], '%d%m%Y').strftime('%d.%m.%Y')

        row_content.append(values[:2])



    df = pd.DataFrame(row_content, columns = rows_headers)

    df['Ticker'] = ticker

    df = df[['Ticker', 'Data', 'Ultimo']]
    
    
    
    return pd.concat([df_historical_values, df])